In [ ]:
# # Mount
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
# Import Library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objs as go
%matplotlib inline

plt.rc('font', family='NanumBarunGothic') 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
all =  pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/all_clean.csv',parse_dates=['STD_YMD'])
dong = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/구_동.xlsx')

In [ ]:
all = pd.merge(dong,all,on='HDONG_NM',how='left')
all['year']=pd.DatetimeIndex(all.STD_YMD).year
gs_list = ['GS_식사', 'GS_간식', 'GS_마실거리', 'GS_홈/리빙', 'GS_헬스/뷰티', 'GS_취미/여가활동','GS_사회활동', 'GS_임신육아']
for i in gs_list:
  all.loc[:,i] = all.loc[:,i].str.strip('%').astype(float) 

In [ ]:
all_seoul = all.query('HDONG_GU_x in ["서울 중구","서울 노원구"]')
gs_col = ['GS_매출지수','GS_식사', 'GS_간식', 'GS_마실거리', 'GS_홈/리빙', 'GS_헬스/뷰티', 'GS_취미/여가활동', 'GS_사회활동', 'GS_임신육아']
gs = all_seoul.groupby(['HDONG_NM','year']).mean().loc[:,gs_col].unstack().reset_index(drop=False).dropna()#상계 8동 na
gs_g = pd.DataFrame(gs.HDONG_NM)
for i in gs_col:
  gs_g[i] = (gs[(i,2020)] - gs[(i,2019)])*100
gs[('GS_매출지수','growth')] = gs_g['GS_매출지수']

In [ ]:
gs = gs.sort_values(by=('GS_매출지수','growth'),ascending=False)
gs.loc[:,['HDONG_NM','GS_매출지수']].head()

# **1. 동별 매출지수 증감률 비교**

In [ ]:
top5 = gs.sort_values(by=('GS_매출지수','growth'),ascending=False).iloc[:5,:].HDONG_NM.tolist()
bott10 = gs.sort_values(by=('GS_매출지수','growth'),ascending=True).iloc[:10,:].HDONG_NM.tolist()
print('top 5:    ',top5)
print('bottom 5: ', bott10)

top 5:     ['상계3.4동', '중계1동', '약수동', '동화동', '상계5동']
bottom 5:  ['광희동', '을지로동', '소공동', '명동', '회현동', '필동', '하계1동', '장충동', '다산동', '월계1동']


In [ ]:
def gr_plot(data,title,color_list):
  plt.figure(figsize = (10,6))
  plt.bar(data.HDONG_NM, data[('GS_매출지수','growth')], color = color_list,width=0.8)
  plt.xlabel('행정동',size=12)
  plt.ylabel('GS_매출지수 증감률(%)',size=12)
  plt.xticks(rotation=40,fontsize=9)
  plt.xlim(-0.5,32.5)
  plt.title(title,fontsize=20)
  plt.show()

all_color = ['#3d89be']*5+['grey']*18+['r']*10
gr_plot(gs,'GS 매출지수 증감률',all_color)

In [ ]:
gs_top = gs.sort_values(by=('GS_매출지수','growth'),ascending=False).iloc[:5,:].loc[:,['HDONG_NM','GS_매출지수']]
gs_bottom = gs.sort_values(by=('GS_매출지수','growth'),ascending=True).iloc[:10,:].loc[:,['HDONG_NM','GS_매출지수']].sort_values(by=('GS_매출지수','growth'),ascending=False)
gs_tb = pd.concat([gs_top,gs_bottom])

In [ ]:
def gr_plot(data,title,color_list):
  plt.figure(figsize = (9,6))
  plt.bar(data.HDONG_NM, data[('GS_매출지수','growth')], color = color_list,width=0.8)
  plt.xlabel('행정동',size=12)
  plt.ylabel('GS_매출지수 증감률(%)',size=12)
  plt.xticks(rotation=35,fontsize=12)
  plt.xlim(-0.5,14.5)
  plt.title(title,fontsize=20)
  plt.show()

tb_color = ['#3d89be']*5+['r']*10
gr_plot(gs_tb,'GS 매출지수 증감률',tb_color)

# **2. GS25 카테고리별 판매지수 증감률**


In [ ]:
all_a =  pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/all_amt.csv',parse_dates=['STD_YMD'])
dong = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/구_동.xlsx')

In [ ]:
all_a = pd.merge(dong,all_a,on='HDONG_NM',how='left')
all_a['year']=pd.DatetimeIndex(all_a.STD_YMD).year
all_a_seoul = all_a.query('HDONG_GU in ["서울 중구","서울 노원구"]')
gs_a_col = ['GS_식사', 'GS_간식', 'GS_마실거리', 'GS_홈/리빙', 'GS_헬스/뷰티', 'GS_취미/여가활동', 'GS_사회활동', 'GS_임신육아']
all_a_seoul
gs_a = all_a_seoul.groupby(['HDONG_NM','year']).sum().loc[:,gs_a_col].unstack().reset_index(drop=False)#.dropna()#상계 8동 na

In [ ]:
gs_a_g = pd.DataFrame(gs_a.HDONG_NM)
for i in gs_a_col:
  cat_growth = (gs_a[(i,2020)] - gs_a[(i,2019)])/gs_a[(i,2019)]
  gs_a_g[i] = cat_growth
  gs_a[(i,'growth')]= cat_growth
gs_a_g['GS_매출지수']=gs.loc[:,['HDONG_NM','GS_매출지수']].sort_values(by='HDONG_NM')[('GS_매출지수','growth')]
gs_a[('GS_매출지수','growth')] = gs_a_g.GS_매출지수

gs_a_g = gs_a_g[gs_a_g.loc(axis=1)['HDONG_NM'].isin(top5 + bott10)].sort_values(by='GS_매출지수',ascending=False)

In [ ]:
gs_a_list = gs_a_g.HDONG_NM.tolist()
print(gs_a_list)
gs_a_rank = gs_a[gs_a.loc(axis=1)['HDONG_NM'].isin(gs_a_list)].sort_values(by=('GS_매출지수','growth'),ascending=False)

In [ ]:
cat='GS_식사'
dat = gs_a_rank[['HDONG_NM',cat]].sort_values(by=(cat,'growth'),ascending=False)
dat_sort_list = dat['HDONG_NM'].tolist()
dat.index = dat['HDONG_NM']
dat_g = pd.DataFrame(dat[[(cat,'growth')]].unstack().reset_index(drop=False).rename({0:'growth'},axis=1).drop(['level_0',	'year'],axis=1))
dat = dat.drop('HDONG_NM',axis=1).drop((cat,'growth'),axis=1).droplevel(0,axis=1).unstack().reset_index(drop=False).rename({0:'매출비중'},axis=1)
dat.max().values[2]+100

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3936: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



6446.813999999997

In [ ]:
def gr_cat_plot_sort(cat, gs_a_rank):

  dat = gs_a_rank[['HDONG_NM',cat]].sort_values(by=(cat,'growth'),ascending=False)
  dat_sort_list = dat['HDONG_NM'].tolist()
  dat.index = dat['HDONG_NM']
  dat_g = pd.DataFrame(dat[[(cat,'growth')]].unstack().reset_index(drop=False).rename({0:'growth'},axis=1).drop(['level_0',	'year'],axis=1))
  dat = dat.drop('HDONG_NM',axis=1).drop((cat,'growth'),axis=1).droplevel(0,axis=1).unstack().reset_index(drop=False).rename({0:'매출비중'},axis=1)

  
  plt.figure(figsize = (12,6))  
  new_cat = cat[3:]
  plt.title(new_cat + ' 매출액 변화 & 증감률',fontsize=20)
  plt.bar(dat[dat.year==2019].index-0.2,dat[dat.year==2019].매출비중,width=0.4, color=['lightsteelblue']*5 + ['mistyrose']*15, label='2019')
  plt.bar(dat[dat.year==2019].index+0.2,dat[dat.year==2020].매출비중,width=0.4, color=['#3d89be']*5 + ['crimson']*15,label='2020')
  plt.xticks(np.array(range(15)),dat[dat.year==2019].HDONG_NM.to_list(),fontsize=12,rotation=35)
  # plt.legend(['2019','2020'],ncol=2,loc='upper center')
  plt.xlabel('행정동',size=12)
  plt.ylabel('GS 매출 비중 * 매출지수',size=12)
  ax2 = plt.twinx()  
  ax2.axhline(y=0, xmin=0, xmax=1,color='grey',linestyle='--')
  ax2.plot('HDONG_NM','growth',data = dat_g,
            color='grey',marker='o',linestyle='--',linewidth=2,markerfacecolor='grey')
  ax2.set_ylabel('증감률(%)',rotation=270)
  ax2.yaxis.labelpad = 20
  ax2.tick_params(axis='y', labelcolor='black')  
    
  plt.show()

# 왼쪽 y = 카테고리별 19,20 비중 & 오른쪽 y = 비중 증감률
### 비중 증감률 단순 sub 해도되는거 맞나 확인!
for i in gs_list:
  gr_cat_plot_sort(i,gs_a_rank)